In [ ]:
!pip install flask flask-ngrok tensorflow pillow
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼
added 22 packages in 3s
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠼

In [ ]:
!pip install pyngrok


In [ ]:
from google.colab import files
import zipfile
import os

# Upload the ZIP file
uploaded = files.upload()  # Choose 'Dataset.zip' or your ZIP file here

# Extract the ZIP file into the correct location
with zipfile.ZipFile('Dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset')


Saving Dataset.zip to Dataset.zip


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


data_dir = 'dataset/Dataset'
img_size = 28
batch_size = 32
epochs = 10


datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=(img_size, img_size),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=(img_size, img_size),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

test_gen = datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(img_size, img_size),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)


assert train_gen.num_classes == 10, f"Expected 10 classes, but got {train_gen.num_classes}"
print("Class indices:", train_gen.class_indices)


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.fit(train_gen, validation_data=val_gen, epochs=epochs)


test_loss, test_acc = model.evaluate(test_gen)
print(f"\n✅ Test Accuracy: {test_acc * 100:.2f}%")


os.makedirs('model', exist_ok=True)
model.save('model/numeral_model.h5')


Found 6895 images belonging to 10 classes.
Found 1969 images belonging to 10 classes.
Found 985 images belonging to 10 classes.
Class indices: {'i': 0, 'ii': 1, 'iii': 2, 'iv': 3, 'ix': 4, 'v': 5, 'vi': 6, 'vii': 7, 'viii': 8, 'x': 9}


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.5868 - loss: 1.2759 - val_accuracy: 0.9167 - val_loss: 0.2729
Epoch 2/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - accuracy: 0.9247 - loss: 0.2541 - val_accuracy: 0.9426 - val_loss: 0.1909
Epoch 3/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - accuracy: 0.9550 - loss: 0.1502 - val_accuracy: 0.9573 - val_loss: 0.1421
Epoch 4/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9715 - loss: 0.0953 - val_accuracy: 0.9639 - val_loss: 0.1260
Epoch 5/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9824 - loss: 0.0635 - val_accuracy: 0.9700 - val_loss: 0.1148
Epoch 6/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - accuracy: 0.9884 - loss: 0.0445 - val_accuracy: 0.9690 - val_loss: 0.1395
Epoch 7/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9886 - loss: 0.0362 - val_accuracy: 0.9711 - val_loss: 0.1003
Epoch 8/10
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9951 - loss: 0.0213 - val_acc


✅ Test Accuracy: 98.58%


In [ ]:
import os
os.makedirs("templates", exist_ok=True)


In [ ]:
%%writefile templates/index.html
<!doctype html>
<html>
  <head>
    <title>Roman Numeral Classifier</title>
  </head>
  <body>
    <h1>Upload an image of a Roman numeral</h1>
    <form method="POST" enctype="multipart/form-data">
      <input type="file" name="file">
      <input type="submit" value="Classify">
    </form>
    {% if prediction %}
      <h2>Prediction: {{ prediction }}</h2>
    {% endif %}
  </body>
</html>


Overwriting templates/index.html


In [ ]:
!ngrok config add-authtoken 2wjhMHpHszkB9ru69V9W4FGOiIj_5VfoH35LMHcimYyVwR51t

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, render_template
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
from PIL import Image


model = tf.keras.models.load_model('model/numeral_model.h5')



class_map = {
    0: 'I',
    1: 'II',
    2: 'III',
    3: 'IV',
    4: 'IX',
    5: 'V',
    6: 'VI',
    7: 'VII',
    8: 'VIII',
    9: 'X'
}

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    if request.method == 'POST':
        file = request.files['file']
        if file:
            img = Image.open(file).convert('L').resize((28, 28))
            img = np.array(img).reshape(1, 28, 28, 1) / 255.0
            pred = model.predict(img)
            pred_class = np.argmax(pred)
            prediction = class_map[pred_class]
    return render_template('index.html', prediction=prediction)


public_url = ngrok.connect(5001)
print(f" * ngrok tunnel URL: {public_url}")


app.run(port=5001)

 * ngrok tunnel URL: NgrokTunnel: "https://73c6-34-118-251-12.ngrok-free.app" -> "http://localhost:5001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/May/2025 05:59:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 05:59:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:16:26] "GET / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:29:45] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:29:50] "POST / HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


INFO:werkzeug:127.0.0.1 - - [07/May/2025 06:30:01] "POST / HTTP/1.1" 200 -
